## A LNMO parameterisation example using PyBOP

This notebook introduces a synthetic re-parameterisation of a single-particle model with corrupted observations. The first step is to import the PyBOP package,

In [ ]:
%pip install git+https://github.com/pybop-team/PyBOP.git@develop -q 
%pip install pybamm -q

Next, we import the added packages and the additional dependancies,

In [ ]:
import pybop
import pybamm
import matplotlib.pyplot as plt
import numpy as np

## Generate Synthetic Data

Next, we need to generate the synthetic data required for later fitting. To do this we will run the PyBaMM forward model and save the generated data. This will be integrated into PyBOP in a future release for fast synthetic generation. First, we define the PyBaMM model,

In [ ]:
model = pybamm.lithium_ion.SPM()
params = model.default_parameter_values

Next, we update the parameters with our selected values and run the simulation.

In [ ]:
params.update(
    {
        "Negative electrode active material volume fraction": 0.52,
        "Positive electrode active material volume fraction": 0.63,
    }
)

Define the experiment and run the forward model to capture the synthetic data.

In [ ]:
experiment = pybamm.Experiment(
    [
        (
            "Discharge at 2C for 5 minutes (1 second period)",
            "Rest for 2 minutes (1 second period)",
            "Charge at 1C for 5 minutes (1 second period)",
            "Rest for 2 minutes (1 second period)",
        ),
    ]
    * 2
)
synthetic = pybamm.Simulation(model, experiment=experiment, parameter_values=params)
solution = synthetic.solve()

Plot the synthetic data,

In [ ]:
synthetic.plot()

## Identify the Parameters

Now, to blind fit the synthetic parameters we define the observation variables.

In [ ]:
observations = [
            pybop.Observed("Time [s]", solution["Time [s]"].data),
            pybop.Observed("Current function [A]", solution["Current [A]"].data),
            pybop.Observed("Voltage [V]", solution["Terminal voltage [V]"].data),
        ]

Next, we define the forward model parameters for fitting.

In [ ]:
fit_params = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        prior=pybop.Gaussian(0.5, 0.02),
        bounds=[0.375, 0.625],
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        prior=pybop.Gaussian(0.65, 0.02),
        bounds=[0.525, 0.75],
    ),
]

Next, we define the parameterisation class. This provides the python methods for parameterisation of the forward models.

In [ ]:
parameterisation = pybop.Parameterisation(
    model, observations=observations, fit_parameters=fit_params
)

Finally, we run the fitting algorithm. For this example, we use a root-mean square cost function with the BOBYQA algorithm implemented in NLOpt

In [ ]:
results, last_optim, num_evals = parameterisation.rmse(
    signal="Voltage [V]", method="nlopt"
)

## Plotting

Re-run forward model with estimated parameters

In [ ]:
params.update(
        {"Electrode height [m]": results[0], 
         "Negative particle radius [m]": results[1], 
         "Positive particle radius [m]":results[2]}
        )
optsol = sim.solve()

Plot the estimated forward model against the synthetic (groundtruth)

In [ ]:
plt.plot(synthetic["Terminal voltage [V]"].data, label='Groundtruth')
plt.plot(optsol["Terminal voltage [V]"].data, label='Estimated')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.legend()
plt.savefig('sol.png')